# Template for segmented MNL models

Sam Maurer, Sep 2018

This notebook tests and demonstrates the segmented MNL models added in [UrbanSim Templates PR #45](https://github.com/UDST/urbansim_templates/pull/45).

In [1]:
import numpy as np
import pandas as pd

import orca
import urbansim_templates

from urbansim_templates import modelmanager
from urbansim_templates.models import SegmentedLargeMultinomialLogitStep

urbansim_templates.__version__

'0.1.dev14'

### Set up testing data

In [2]:
d1 = {'oid': np.arange(10), 
      'group': np.random.choice(['A','B','C'], size=10),
      'obsval': np.random.random(10),
      'choice': np.random.choice(np.arange(20), size=10)}

d2 = {'aid': np.arange(20), 
      'altval': np.random.random(20)}

obs = pd.DataFrame(d1).set_index('oid')
orca.add_table('obs', obs)

alts = pd.DataFrame(d2).set_index('aid')
orca.add_table('alts', alts)

In [3]:
obs

,group,obsval,choice
oid,,,
0,B,0.601075,0
1,B,0.770388,0
2,A,0.251365,19
3,A,0.538712,12
4,C,0.013998,13
5,B,0.825602,6
6,C,0.036147,14
7,A,0.215300,7
8,A,0.354720,1


### Test model features

In [4]:
m = SegmentedLargeMultinomialLogitStep()

In [5]:
m.defaults.choosers = 'obs'
m.defaults.alternatives = 'alts'
m.defaults.choice_column = 'choice'
m.defaults.model_expression = 'obsval + altval'
m.segmentation_column = 'group'

In [6]:
m.build_submodels()

Building submodels for 3 categories: ['A' 'B' 'C']


In [7]:
for k, v in m.submodels.items():
    print(v.chooser_filters)

group == 'A'
group == 'B'
group == 'C'


In [8]:
# Change one model expression

m.submodels['A'].model_expression = 'altval'

for k, v in m.submodels.items():
    print(v.model_expression)

altval
obsval + altval
obsval + altval


In [9]:
# Reset the model expressions

m.defaults.model_expression = 'obsval + altval'

for k, v in m.submodels.items():
    print(v.model_expression)

obsval + altval
obsval + altval
obsval + altval


In [10]:
m.fit_all()

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:              4
Model:         Multinomial Logit   Df Residuals:                  1
Method:       Maximum Likelihood   Df Model:                      3
Date:                 2018-09-21   Pseudo R-squ.:             0.000
Time:                      16:03   Pseudo R-bar-squ.:        -0.255
AIC:                      29.555   Log-Likelihood:          -11.778
BIC:                      27.714   LL-Null:                 -11.778
               coef   std err         z     P>|z|   Conf. Int.
--------------------------------------------------------------
Intercept   -0.0000     1.684    -0.000     1.000             
obsval      -0.0000     4.147    -0.000     1.000             
altval      -0.0231     2.002    -0.012     0.991             
                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:              3
Model

/Users/maurer/Dropbox/Git-imac/udst/choicemodels/choicemodels/mnl.py:235: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  chosen = np.reshape(self._df[[self._choice_col]].as_matrix(),
/Users/maurer/Dropbox/Git-imac/udst/choicemodels/choicemodels/mnl.py:238: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  log_lik, fit = mnl_estimate(model_design.as_matrix(), chosen, self._numalts)
